# Some timeseries scenarios

In [1]:
import psycopg2 as psy
from datetime import datetime, timedelta
import json

from osgeo import ogr
from osgeo import osr

import numpy as np
import tiledb
import os


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Setting thing up

### Setup databases

In [2]:
# create testdb
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

con = psy.connect("dbname=postgres host=timescaledb user=postgres password=foobar")

with con:
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:
        cur.execute("DROP DATABASE IF EXISTS testdb;")
        cur.execute('CREATE DATABASE testdb')

# not sure if the AUTOCOMMIT is needed
with con:
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:   
        cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb CASCADE;")
con.close()

In [3]:
con = psy.connect("dbname=testdb host=timescaledb user=postgres password=foobar")

with con:
    with con.cursor() as cur:   
        cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
        cur.execute("SELECT postgis_version();")
        print(cur.fetchall())
        cur.execute("SELECT srid, auth_name, proj4text FROM spatial_ref_sys LIMIT 3;")
        for x in cur.fetchall():
            print(x)
        cur.execute("SELECT srid, auth_name, proj4text FROM spatial_ref_sys WHERE srid = 3003")
        print(cur.fetchall())
con.close()

[('2.5 USE_GEOS=1 USE_PROJ=1 USE_STATS=1',)]
(3819, 'EPSG', '+proj=longlat +ellps=bessel +towgs84=595.48,121.69,515.35,4.115,-2.9383,0.853,-3.408 +no_defs ')
(3821, 'EPSG', '+proj=longlat +ellps=aust_SA +no_defs ')
(3824, 'EPSG', '+proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs ')
[(3003, 'EPSG', '+proj=tmerc +lat_0=0 +lon_0=9 +k=0.9996 +x_0=1500000 +y_0=0 +ellps=intl +towgs84=-104.1,-49.1,-9.9,0.971,-2.917,0.714,-11.68 +units=m +no_defs ')]


### Prepare a generic sensor table

In [4]:
source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(3003)

transform = osr.CoordinateTransformation(source, target)

def map_to_monte_mario(wkt):
    geom = ogr.CreateGeometryFromWkt(wkt)
    geom.Transform(transform)
    return geom.ExportToWkt()

def create_sensors_index_table(conn):
    query = """
        CREATE TABLE sensors_index (
            id int4 primary key,
            name VARCHAR(20),
            type VARCHAR(20),
            category VARCHAR(20),
            geom geometry,
            precision REAL
        );
    """
    with conn:
        with conn.cursor() as cur:
            cur.execute(query)

In [5]:
con = psy.connect("dbname=testdb host=timescaledb user=postgres password=foobar")
create_sensors_index_table(con)

## Manage point sensors

In [6]:
class point_sensor_description:
    def __init__(self, stype, sensors):
        self.desc = {'type': stype, 'category': ["meteosensor"]}
        self.desc['controlledProperty'] = sensors
        self.desc['footprint'] = {'fixed': True, 'geometry': 'point3D'}
        
    def to_json(self):
        return json.dumps(self.desc)

In [7]:
def create_point_sensor_table(conn, desc):
    conv = {'float32': 'REAL', 'int32': 'INTEGER'}
    def create_field(k, v):
        return "{} {} NULL".format(k, conv[v])
    fields = ',\n'.join(create_field(k, v) for k, v in desc['controlledProperty'])
    SQL = """
    CREATE TABLE {} (
       time     TIMESTAMPTZ   NOT NULL,
       id       int4          NOT NULL,
       {}
    );
    """.format(desc['type'], fields)
    with conn:
        with conn.cursor() as cur:
            cur.execute(SQL)
    SQL = "SELECT create_hypertable('{}', 'time');".format(desc['type'])
    with conn:
        with conn.cursor() as cur:
            cur.execute(SQL)

def register_point_sensor(conn, sid, stype, sname, scategory, longitude, latitude):
    SQL = """INSERT INTO sensors_index (id, name, type, category, geom, precision)
             VALUES (%s, %s, %s, %s, ST_GeomFromText(%s, 3003), 10);"""
    with conn:
        with conn.cursor() as cur:
            cur.execute(SQL,
                        (sid, stype, sname, scategory, 
                         map_to_monte_mario("POINT (%s %s)" % (longitude, latitude))))
            
def insert_point_sensor_data(conn, desc, sid, values):
    SQL = "INSERT INTO {} (time, id, {}) VALUES (%s, {}, {})".format(
        desc['type'], ", ".join(k for k, v in desc['controlledProperty']),
        sid, ", ".join(["%s"] * len(desc['controlledProperty'])))
    with conn:
        for v in values:
            with conn.cursor() as cur:
                cur.execute(SQL, v)

This is the description of the point_sensor class 'meteo_station1'.

In [8]:
sd = point_sensor_description('meteo_station1', 
         sensors=[('temperature', 'float32'), ('pressure', 'float32'), ('humidity', 'float32')],
     )

In [9]:
sd.desc

{'type': 'meteo_station1',
 'category': ['meteosensor'],
 'controlledProperty': [('temperature', 'float32'),
  ('pressure', 'float32'),
  ('humidity', 'float32')],
 'footprint': {'fixed': True, 'geometry': 'point3D'}}

Register two sensors of the 'meteo_station1' class and create a table to store data from that class of sensors.

In [10]:
register_point_sensor(con, 167, 'meteo_station1', 'rupert001', 'meteosensor', longitude=9.121661, latitude=39.224841)
register_point_sensor(con, 191, 'meteo_station1', 'rupert002', 'meteosensor', longitude=9.121661, latitude=39.223971)
create_point_sensor_table(con, sd.desc)

List all currently known sensors.

In [11]:
with con:
    with con.cursor() as cur:
        cur.execute("SELECT id, ST_AsText(ST_Transform(geom,4326)) from sensors_index;")
        for x in cur.fetchall():
            print(x)

(167, 'POINT(9.12166100720147 39.2248410015303)')
(191, 'POINT(9.12166100720134 39.2239710015304)')


Insert data associated to the sensors.

In [12]:
def generate_time_series(tstart, delta, n):
    return [(tstart + i * delta, np.random.normal(283, 10), np.random.normal(1050, 30), np.random.normal(0.35, 0.1))
            for i in range(n)]

In [13]:
insert_point_sensor_data(con, sd.desc, 167, generate_time_series(datetime.now(), timedelta(seconds=60), 10))
insert_point_sensor_data(con, sd.desc, 191, generate_time_series(datetime.now(), timedelta(seconds=60), 10))

In [14]:
with con:
    with con.cursor() as cur:
        cur.execute("SELECT id, time, temperature from meteo_station1 where id = 167 LIMIT 3;")
        for x in cur.fetchall():
            print(x)

(167, datetime.datetime(2019, 3, 13, 17, 35, 46, 913789, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 288.341)
(167, datetime.datetime(2019, 3, 13, 17, 36, 46, 913789, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 293.542)
(167, datetime.datetime(2019, 3, 13, 17, 37, 46, 913789, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 270.159)


Find all sensors within 100 meters from a given point.

In [15]:
with con:
    with con.cursor() as cur:
        cur.execute("""
        SELECT id, ST_AsText(ST_Transform(geom,4326)) from sensors_index
        WHERE ST_DWithin(geom, ST_GeomFromText(%s, 3003), 100);
        """, (map_to_monte_mario('POINT(9.121661 39.223841)'), ))
        for x in cur.fetchall():
            print(x)

(191, 'POINT(9.12166100720134 39.2239710015304)')


Find all sensors within 1000 meters from a given point.

In [16]:
with con:
    with con.cursor() as cur:
        cur.execute("""
        SELECT id, ST_AsText(ST_Transform(geom,4326)) from sensors_index
        WHERE ST_DWithin(geom, ST_GeomFromText(%s, 3003), 1000);
        """, (map_to_monte_mario('POINT(9.121661 39.223841)'), ))
        for x in cur.fetchall():
            print(x)

(167, 'POINT(9.12166100720147 39.2248410015303)')
(191, 'POINT(9.12166100720134 39.2239710015304)')


Completely gratuituos experiment with folium.

In [17]:
# more or less direct cut&paste from a folium example.
import folium
import folium.plugins
map_1 = folium.Map(location=[39.223841,  9.121661], zoom_start=17,tiles=None)
map_1.add_tile_layer()
data_sites = [{"status": "is_active", "coordinates" : [39.224841, 9.121661]},
              {"status": "is_active", "coordinates" : [39.223971, 9.121661]},
              {"status": "not_active", "coordinates" : [39.223824, 9.121661]}, 
              {"status": "not_active", "coordinates" : [39.223810, 9.121661]}]
feature_group_active = folium.FeatureGroup(name='Active')
feature_group_unactive = folium.FeatureGroup(name='Unactive')
marker_cluster_active = folium.plugins.MarkerCluster()
marker_cluster_unactive =folium.plugins.MarkerCluster()
for site in data_sites:
    if(site["status"]=="is_active"):
        marker_active = folium.Marker(site["coordinates"],popup="OK",
                                      icon = folium.Icon(color='green',icon='ok-sign'))
        marker_cluster_active.add_child(marker_active)
    else:
        marker_unactive = folium.Marker(site["coordinates"],popup="KO",
                                        icon = folium.Icon(color='red',icon='exclamation-sign'))
        marker_cluster_unactive.add_child(marker_unactive)
feature_group_active.add_child(marker_cluster_active)
feature_group_unactive.add_child(marker_cluster_unactive)
map_1.add_child(feature_group_active)
map_1.add_child(feature_group_unactive)

## Manage area sensors

In [18]:
class area_sensor_description:
    def __init__(self, stype, sensors):
        self.desc = {'type': stype, 'category': ["meteosensor"]}
        self.desc['controlledProperty'] = sensors
        self.desc['footprint'] = {'fixed': True, 'geometry': 'polygon'}
        
    def to_json(self):
        return json.dumps(self.desc)

In [19]:
def create_area_sensor_table(conn, desc):
    SQL = """
    CREATE TABLE {} (
       time     TIMESTAMPTZ   NOT NULL,
       id       int4          NOT NULL,
       dataset  TEXT          NULL,
       frame    int4          NULL
    );
    """.format(desc['type'])
    with conn:
        with conn.cursor() as cur:
            cur.execute(SQL)
    SQL = "SELECT create_hypertable('{}', 'time');".format(desc['type'])
    with conn:
        with conn.cursor() as cur:
            cur.execute(SQL)


def register_area_sensor(conn, sid, stype, sname, scategory, polygon):
    polygon = "POLYGON(({}))".format(", ".join("{} {}".format(lon, lat) for lon, lat in polygon))
    SQL = """INSERT INTO sensors_index (id, name, type, category, geom, precision)
             VALUES (%s, %s, %s, %s, ST_GeomFromText(%s, 3003), 10);"""
    with conn:
        with conn.cursor() as cur:
            cur.execute(SQL,
                        (sid, stype, sname, scategory, 
                         map_to_monte_mario(polygon)))


### Create data arrays

In [20]:
#ctx = tiledb.Ctx({'vfs.hdfs.username': 'root'})
ctx = tiledb.Ctx()

In [22]:
#top = tiledb.group_create("hdfs://hdfs:9000/radar01", ctx)
top = tiledb.group_create("radar04")

In [23]:
coords = tiledb.group_create(os.path.join(top, "coords"), ctx)

In [24]:
tdim = tiledb.Dim('time', domain=(0, 60*24*365*10), dtype=np.int32, tile=60)
xdim = tiledb.Dim('x', domain=(0, 1023), dtype=np.int32, tile=4)
ydim = tiledb.Dim('y', domain=(0, 1023), dtype=np.int32, tile=5)

tschema = tiledb.ArraySchema(domain=tiledb.Domain(tdim), 
                             sparse=False, 
                             attrs=[tiledb.Attr(name='T', dtype=np.float64)]) # seconds from reference
xschema = tiledb.ArraySchema(domain=tiledb.Domain(xdim), 
                             sparse=False, 
                             attrs=[tiledb.Attr(name='X', dtype=np.float32)]) # monte mario X coordinates of point
yschema = tiledb.ArraySchema(domain=tiledb.Domain(ydim), 
                             sparse=False, 
                             attrs=[tiledb.Attr(name='Y', dtype=np.float32)]) # monte mario Y coordinates of point
tiledb.DenseArray.create(os.path.join(coords, "time"), tschema)
tiledb.DenseArray.create(os.path.join(coords, "X"), xschema)
tiledb.DenseArray.create(os.path.join(coords, "Y"), yschema)

dom = tiledb.Domain(tdim, xdim, ydim)
p = tiledb.Attr(name='precipitation', dtype=np.float32)
schema = tiledb.ArraySchema(domain=dom, sparse=False, attrs=[p])
tiledb.DenseArray.create(os.path.join(top, "variables"), schema) # it does not map directly to netcdf4 

In [25]:
asd = area_sensor_description('radar_mark0', ['precipitation'])

In [26]:
asd.desc

{'type': 'radar_mark0',
 'category': ['meteosensor'],
 'controlledProperty': ['precipitation'],
 'footprint': {'fixed': True, 'geometry': 'polygon'}}

In [27]:
create_area_sensor_table(con, asd.desc)

In [28]:
polygon = [(8.752247222222222, 39.5049278), (8.75418611111111, 38.9513111), (9.463199999999999, 38.9506528),
           (9.466852777777778, 39.5042528), (8.752247222222222, 39.5049278)]

In [29]:
register_area_sensor(con, 200, 'radar_mark0', 'radar@unica', 'meteosensor', polygon)

Check what we have registered.

In [30]:
with con:
    with con.cursor() as cur:
        cur.execute("SELECT id, ST_AsText(ST_Transform(geom,4326)) from sensors_index;")
        for x in cur.fetchall():
            print(x)

(167, 'POINT(9.12166100720147 39.2248410015303)')
(191, 'POINT(9.12166100720134 39.2239710015304)')
(200, 'POLYGON((8.75224722959804 39.5049278015358,8.7541861184325 38.951311101573,9.46320000703916 38.9506528015271,9.46685278486905 39.504252801489,8.75224722959804 39.5049278015358))')


In [31]:
with con:
    with con.cursor() as cur:
        cur.execute("""
        SELECT id, ST_AsText(ST_Transform(geom,4326)) from sensors_index
        WHERE ST_DWithin(geom, ST_GeomFromText(%s, 3003), 1000);
        """, (map_to_monte_mario('POINT(9.121661 39.223841)'), ))
        for x in cur.fetchall():
            print(x)

(167, 'POINT(9.12166100720147 39.2248410015303)')
(191, 'POINT(9.12166100720134 39.2239710015304)')
(200, 'POLYGON((8.75224722959804 39.5049278015358,8.7541861184325 38.951311101573,9.46320000703916 38.9506528015271,9.46685278486905 39.504252801489,8.75224722959804 39.5049278015358))')


### Insert some radar data

This is a rather basic example. 

In [32]:
def insert_area_sensor_data_sql(conn, tstamp, sid, dataset, frame):
    SQL = "INSERT INTO radar_mark0 (time, id, dataset, frame) VALUES ('{}', {}, '{}', {})".format(
        tstamp, sid, dataset, frame)
    with conn:
        with conn.cursor() as cur:
            cur.execute(SQL)

def insert_area_sensor_data_tiledb(ctx, delta_sec, dataset, idx, data):
    var = os.path.join(dataset, 'variables')
    time = os.path.join(dataset, 'coords/time')
    with tiledb.DenseArray(time, mode='w') as T:
        T[idx:(idx+1)] = delta_sec
    print('saved T')
    with tiledb.DenseArray(var, mode='w') as A:
        A[idx:(idx+1), ...] = data
    #

def insert_area_sensor_data(conn, ctx, sid, tstamp, frame, data):
    dataset = top
    delta_sec = (tstamp - start).total_seconds()
    print('ready to save array')
    insert_area_sensor_data_tiledb(ctx, delta_sec, dataset, frame, data)
    print('ready to save sql')
    insert_area_sensor_data_sql(conn, tstamp, sid, dataset, frame)

In [33]:
start = datetime.now()

In [34]:
data = np.zeros((1024, 1024), dtype=np.float32)

In [35]:
insert_area_sensor_data(con, ctx, 200, start + timedelta(seconds=60), 0, data)

ready to save array
saved T
ready to save sql
